In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

# Set the seed for reproducibility
torch.manual_seed(42)

In [ ]:
# Define transformations for the training and testing sets
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Download and load the training dataset
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=100, shuffle=True, num_workers=2)

# Download and load the testing dataset
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)

# Get the number of images in the training dataset
print(f'Number of images in the training dataset: {len(trainset)}')

# Get the size of each image (height, width, channels)
image, label = trainset[0]
print(f'Size of each image: {image.size()}')  # Should print torch.Size([3, 32, 32])


Files already downloaded and verified
Files already downloaded and verified
Number of images in the training dataset: 50000
Size of each image: torch.Size([3, 32, 32])


In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # First convolutional layer: 3 input channels (RGB), 6 output channels, 5x5 kernel
        self.conv1 = nn.Conv2d(3, 6, 5)
        # Max pooling layer: 2x2 kernel, stride 2
        self.pool = nn.MaxPool2d(2, 2)
        # Second convolutional layer: 6 input channels, 16 output channels, 5x5 kernel
        self.conv2 = nn.Conv2d(6, 16, 5)
        # Fully connected layers
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        # Apply the first convolution, ReLU, and max pooling
        x = self.pool(torch.relu(self.conv1(x)))
        # Apply the second convolution, ReLU, and max pooling
        x = self.pool(torch.relu(self.conv2(x)))
        # Flatten the output
        x = x.view(-1, 16 * 5 * 5)
        # Apply the first fully connected layer, ReLU
        x = torch.relu(self.fc1(x))
        # Apply the second fully connected layer, ReLU
        x = torch.relu(self.fc2(x))
        # Apply the third fully connected layer (output layer)
        x = self.fc3(x)
        return x

# Initialize the network
net = Net()

In [ ]:
total_params = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(f'Total number of parameters in the network: {total_params}')


Total number of parameters in the network: 62006


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)

In [ ]:
# Get a batch of training data
dataiter = iter(trainloader)
images, labels = next(dataiter)

# Perform a forward pass
outputs = net(images)
initial_loss = criterion(outputs, labels)
print(f'Initial loss: {initial_loss.item()}')

Initial loss: 2.3038909435272217


In [ ]:
for epoch in range(2):  # loop over the dataset twice
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = net(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Print statistics
        running_loss += loss.item()
        if i % 100 == 99:  # print every 100 mini-batches
            print(f'[Epoch {epoch + 1}, Batch {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')

[Epoch 1, Batch 100] loss: 2.298
[Epoch 1, Batch 200] loss: 2.116
[Epoch 1, Batch 300] loss: 1.882
[Epoch 1, Batch 400] loss: 1.695
[Epoch 1, Batch 500] loss: 1.591
[Epoch 2, Batch 100] loss: 1.548
[Epoch 2, Batch 200] loss: 1.472
[Epoch 2, Batch 300] loss: 1.457
[Epoch 2, Batch 400] loss: 1.437
[Epoch 2, Batch 500] loss: 1.378
Finished Training


In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f'Accuracy of the network on the 10000 test images: {accuracy:.2f}%')

Accuracy of the network on the 10000 test images: 50.93%
